# How To Use Cloud-Config For Your Initial Server Setup
[How To Use Cloud-Config For Your Initial Server Setup](https://www.digitalocean.com/community/tutorials/how-to-use-cloud-config-for-your-initial-server-setup)

## Setting Up the New User Account

The first thing we need to do is configure our new user account.

This is where almost all of the work will take place. The root account has no password by default, so we do not have to “unset” any password there.

### Create the New User

To create a new user, we use the `users` directive. This will be contain a list of all of the new accounts that we want to create. Since we are only creating a single account, we will have a list of one. To follow the guide we linked to, we will call this new account `demo`.\
要创建一个新用户，我们使用`users`指令。这将包含我们想要创建的所有新帐户的列表。因为我们只创建一个帐户，所以我们将有一个帐户列表。按照我们链接到的指南，我们将称这个新帐户为`demo`。

Remember, we must start out our `cloud-config` files with `#cloud-config` alone on the first line. So far, our file will look like this:\
记住，我们必须在`cloud-config`文件的第一行单独使用`#cloud-config`。到目前为止，我们的文件是这样的:
```shell
#cloud-config
users:
  - name: demo
```

If we wanted to add additional users, we could do so by placing an item below and horizontally aligned with this one, beginning with a dash again, like this:\
如果我们想添加额外的用户，我们可以在下面放置一个项目，并与这个水平对齐，再次以破折号开始，像这样:
```shell
#cloud-config
users:
  - name: demo
  - name: second_user
```
Each of the dashes indicates a separate user account under which we can add the user’s details (which we will be doing momentarily). We are only creating a single user though, so we won’t have this second line in this guide.\
每个破折号都表示一个单独的用户帐户，我们可以在该帐户下添加用户的详细信息(我们马上就会这么做)。不过，我们只创建了一个用户，所以在本指南中不会有第二行。

### Adding Authorized Keys

In order to log into this new account without a password, we will need to provide one or more of our SSH public keys. These will be added to the new user’s `authorized_keys` file within the `.ssh` directory in their home directory.\
为了在没有密码的情况下登录这个新帐户，我们需要提供一个或多个SSH公钥。这些文件将被添加到新用户主目录下的`.ssh`目录下的`authorized_keys`文件中。

This is accomplished with the `ssh-authorized-keys` directive, which is a sub-item to a `users` entry. Basically this means that we align it with our `name` directive, but do not give it a dash, since it is not the start of a new user entry.\
这是通过`ssh-authorized-keys`指令完成的，它是`users`条目的子项。基本上，这意味着我们将它与我们的`name`指令对齐，但不给它一个破折号，因为它不是一个新的用户条目的开始。

The `ssh-authorized-keys` entry actually takes a list of keys. This allows you to add more than one SSH public key to the file. For instance, if you have an SSH key pair for your laptop, your desktop, and your computer at work, you could add all of those as separate items in the `ssh-authorized-keys` list.

To get the contents of your public key of your local computer, you can type:

```shell
cat ~/.ssh/id_rsa.pub
```
```shell
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCv60WjxoM39LgPDbiW7ne3gu18q0NIVv0RE6rDLNal1quXZ3nqAlANpl5qmhDQ+GS/sOtygSG4/9aiOA4vXO54k1mHWL2irjuB9XbXr00+44vSd2q/vtXdGXhdSMTf4/XK17fjKSG/9y3yD6nml6q9XgQxx9Vf/IkaKdlK0hbC1ds0+8h83PTb9dF3L7hf3Ch/ghvj5++tWJFdFeG+VI7EDuKNA4zL8C5FdYYWFA88YAmM8ndjA5qCjZXIIeZvZ/z9Kpy6DL0QZ8T3NsxRKapEU3nyiIuEAmn8fbnosWcsovw0IS1Hz6HsjYo4bu/gA82LWt3sdRUBZ/7ZsVD3ELip user@example.com
```

You can then paste the complete contents as an item under our `ssh-authorized-keys` entry. SSH public keys can be posted openly, so this does not represent a security risk:
```shell
#cloud-config
users:
  - name: demo
    ssh-authorized-keys:
      - ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCv60WjxoM39LgPDbiW7ne3gu18q0NIVv0RE6rDLNal1quXZ3nqAlANpl5qmhDQ+GS/sOtygSG4/9aiOA4vXO54k1mHWL2irjuB9XbXr00+44vSd2q/vtXdGXhdSMTf4/XK17fjKSG/9y3yD6nml6q9XgQxx9Vf/IkaKdlK0hbC1ds0+8h83PTb9dF3L7hf3Ch/ghvj5++tWJFdFeG+VI7EDuKNA4zL8C5FdYYWFA88YAmM8ndjA5qCjZXIIeZvZ/z9Kpy6DL0QZ8T3NsxRKapEU3nyiIuEAmn8fbnosWcsovw0IS1Hz6HsjYo4bu/gA82LWt3sdRUBZ/7ZsVD3ELip user@example.com
```

If you wanted to add additional keys, that can be done by adding another dash followed by the second public key:
```shell
#cloud-config
users:
  - name: demo
    ssh-authorized-keys:
      - key_one
      - key_two
```
Add any keys that you wish to use to log into this account here.

### Set Up Sudo Access

The next step is to configure `sudo` access to our new account. To reiterate, we will be configuring password-less `sudo` access since we will not be setting a password on this account due to the security limitations.\
下一步是配置`sudo`访问我们的新帐户。重申一下，我们将配置无密码的`sudo`访问，因为由于安全限制，我们不会在此帐户上设置密码。

To configure access, we will actually be doing two separate steps.\
要配置访问，我们将实际执行两个单独的步骤。

First, we will be creating the entry we want to use for the `sudoers` file. Our changes will actually be written to a separate file in the `/etc/sudoers.d` directory, which `/etc/sudoers` includes when parsed.\
首先，我们将创建我们想要用于`sudoers`文件的条目。我们的更改实际上会被写入`/etc/sudoers.d`目录中的一个单独的文件，当解析时`/etc/sudoers`包含这个文件。

The entry we need to create will not need to include the username, since `cloud-init` is smart enough to figure out the account name from the entry information. The directive we need to use is `sudo`, which is aligned with our other `users` level directives.\
我们需要创建的条目不需要包含用户名，因为`cloud-init`足够聪明，可以从条目信息中找出帐户名称。我们需要使用的指令是`sudo`，它与其他`users`级别的指令保持一致。

For our guide, since we are configuring password-less `sudo` capability, will look like this:\
对于我们的指南，因为我们正在配置无密码的`sudo`功能，将会像这样:
```shell
#cloud-config
users:
  - name: demo
    ssh-authorized-keys:
      - ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCv60WjxoM39LgPDbiW7ne3gu18q0NIVv0RE6rDLNal1quXZ3nqAlANpl5qmhDQ+GS/sOtygSG4/9aiOA4vXO54k1mHWL2irjuB9XbXr00+44vSd2q/vtXdGXhdSMTf4/XK17fjKSG/9y3yD6nml6q9XgQxx9Vf/IkaKdlK0hbC1ds0+8h83PTb9dF3L7hf3Ch/ghvj5++tWJFdFeG+VI7EDuKNA4zL8C5FdYYWFA88YAmM8ndjA5qCjZXIIeZvZ/z9Kpy6DL0QZ8T3NsxRKapEU3nyiIuEAmn8fbnosWcsovw0IS1Hz6HsjYo4bu/gA82LWt3sdRUBZ/7ZsVD3ELip user@example.com
    sudo: ['ALL=(ALL) NOPASSWD:ALL']
```

This will create a file in the `/etc/sudoers.d` directory called `90-cloud-init-users`. Inside of this file, the entry will look like this:
```shell
demo ALL=(ALL) NOPASSWD:ALL
```

### Set the Shell Environment

By default, newly created users have their default shell set to the very basic `/bin/sh` shell.\
默认情况下，新创建的用户的默认shell设置为非常基本的`/bin/sh`shell。

This is a much more pared down environment than most people are used to, so we want to manually specify a `bash` shell environment for our new user.\
这是一个比大多数人所习惯的环境要精简得多的环境，因此我们希望为我们的新用户手动指定一个`bash` shell环境。

This can be accomplished with the `shell` directive within the `users` level item. All we need to do is point it to the full path of the `bash` executable:\
这可以通过`users`级项目中的`shell`指令来实现。我们所需要做的就是将它指向`bash`可执行文件的完整路径:

```shell
#cloud-config
users:
  - name: demo
    ssh-authorized-keys:
      - ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCv60WjxoM39LgPDbiW7ne3gu18q0NIVv0RE6rDLNal1quXZ3nqAlANpl5qmhDQ+GS/sOtygSG4/9aiOA4vXO54k1mHWL2irjuB9XbXr00+44vSd2q/vtXdGXhdSMTf4/XK17fjKSG/9y3yD6nml6q9XgQxx9Vf/IkaKdlK0hbC1ds0+8h83PTb9dF3L7hf3Ch/ghvj5++tWJFdFeG+VI7EDuKNA4zL8C5FdYYWFA88YAmM8ndjA5qCjZXIIeZvZ/z9Kpy6DL0QZ8T3NsxRKapEU3nyiIuEAmn8fbnosWcsovw0IS1Hz6HsjYo4bu/gA82LWt3sdRUBZ/7ZsVD3ELip user@example.com
    sudo: ['ALL=(ALL) NOPASSWD:ALL']
    groups: sudo
    shell: /bin/bash
```
We have completed our new user configuration. We can now move on to the optional steps that lock down our SSH daemon.